In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split

Pisahkan keseluruhan data menjadi data pelatihan sebanyak 75% dan data tes 
sebanyak 25%. Baik untuk data pelatihan dan data tes harus dibagi secara proporsional 
untuk masing-masing kelas label menggunakan *stratified sampling*. Tunjukkan statistik 
dari hasil pemisahan data tersebut.

In [13]:
train_df = pd.read_csv('app_train_cleaned_encoded.csv')
test_df = pd.read_csv('app_test_cleaned_encoded.csv')

print(train_df['TARGET'])
print(test_df.columns)

0        1.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
48739    0.0
48740    0.0
48741    0.0
48742    0.0
48743    0.0
Name: TARGET, Length: 48744, dtype: float64
Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE',
       ...
       'HOUSETYPE_MODE_block of flats', 'HOUSETYPE_MODE_specific housing',
       'HOUSETYPE_MODE_terraced house', 'WALLSMATERIAL_MODE_Block',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden'],
      dtype='object', length=236)


In [14]:

# Pisahkan fitur dan label
X = train_df.drop(columns=['SK_ID_CURR', 'TARGET'])
y = train_df['TARGET']


# Split data menjadi train dan test 75%-25% dengan stratified sampling
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25, 
    stratify=y, 
    random_state=42
)

# Cek statistik hasil split
print("Jumlah data sebelum split:")
print(y.value_counts(normalize=True))
print()

print("Jumlah data setelah split:")
print("Train set:")
print(y_train.value_counts(normalize=True))
print()
print("Test set:")
print(y_test.value_counts(normalize=True))

Jumlah data sebelum split:
TARGET
0.0    0.919436
1.0    0.080564
Name: proportion, dtype: float64

Jumlah data setelah split:
Train set:
TARGET
0.0    0.919443
1.0    0.080557
Name: proportion, dtype: float64

Test set:
TARGET
0.0    0.919416
1.0    0.080584
Name: proportion, dtype: float64


In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

Guna memperoleh model klasifikasi terbaik untuk beberapa metode klasifikai yang harus 
diimplementasikan, gunakan keseluruhan data pelatihan menggunakan *10-fold cross 
validation*. Gunakan metode stratified sampling dalam menentukan pembagian data 
dalam 10-fold cross validation. 

In [16]:
# Set up 10-fold stratified cross-validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Kfold splitter
strat_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=130)

In [17]:
train_set_kfold = []
test_set_kfold = []

for fold, (train_index, test_index) in enumerate(kfold.split(X,y)):
    print(f"Fold {fold}")
    print(f"  Train: index={train_index[:5]}... ({len(train_index)} rows)")
    print(f"  Test:  index={test_index[:5]}... ({len(test_index)} rows)")
    train_set_kfold.append(train_index)
    test_set_kfold.append(test_index)


Fold 0
  Train: index=[0 2 3 4 6]... (43869 rows)
  Test:  index=[ 1  5 24 36 54]... (4875 rows)
Fold 1
  Train: index=[0 1 2 3 4]... (43869 rows)
  Test:  index=[10 13 38 46 52]... (4875 rows)
Fold 2
  Train: index=[0 1 2 3 4]... (43869 rows)
  Test:  index=[17 21 34 37 39]... (4875 rows)
Fold 3
  Train: index=[1 3 4 5 6]... (43869 rows)
  Test:  index=[ 0  2  8 31 50]... (4875 rows)
Fold 4
  Train: index=[0 1 2 3 4]... (43870 rows)
  Test:  index=[ 9 11 14 19 33]... (4874 rows)
Fold 5
  Train: index=[0 1 2 3 4]... (43870 rows)
  Test:  index=[20 30 48 70 76]... (4874 rows)
Fold 6
  Train: index=[0 1 2 3 4]... (43870 rows)
  Test:  index=[18 22 29 32 40]... (4874 rows)
Fold 7
  Train: index=[0 1 2 3 5]... (43870 rows)
  Test:  index=[ 4  6 15 16 25]... (4874 rows)
Fold 8
  Train: index=[0 1 2 4 5]... (43870 rows)
  Test:  index=[ 3 23 45 67 71]... (4874 rows)
Fold 9
  Train: index=[0 1 2 3 4]... (43870 rows)
  Test:  index=[ 7 12 27 35 64]... (4874 rows)


SVM

In [18]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [19]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

Eksperimen untuk memperoleh model klasifikasi SVM terbaik menggunakan kombinasi dua jenis kernel: linear dan radial basis function (RBF) serta lima parameter C yang dapat anda tentukan sendiri dalam eksperimen. Untuk ini secara total akan terdapat 10 kombinasi nilai parameter kernel dan C. 

Untuk setiap komibinasi nilai seperti disebutkan dalam langkah pertama, jalankan model klasifikasi SVM menggunakan 10-cros validation. Untuk ini, setiap kombinasi nilai 
parameter SVM akan memberikan 10 hasil validasi dari model. 

Hitung akurasi, presisi, recall, f1-score, dan AUC ROC dari setiap data validasi dari langkah ke-2.  


In [ ]:
kernel_list = ['linear', 'rbf']
C_values = [0.1, 1, 10, 100, 1000]

parameter_grid = [(k, c) for k in kernel_list for c in C_values]

In [21]:
def evaluate_svm(kernel, C, train_idx, val_idx, X_train, y_train):
    X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Buat dan fit model SVM
    svm_model = SVC(kernel=kernel, C=C, probability=True, random_state=42)
    svm_model.fit(X_fold_train, y_fold_train)
    
    # Prediksi
    y_pred = svm_model.predict(X_fold_val)
    y_proba = svm_model.predict_proba(X_fold_val)[:,1]
    
    # Hitung metrik
    metrics = {
        'accuracy': accuracy_score(y_fold_val, y_pred),
        'precision': precision_score(y_fold_val, y_pred, zero_division=0),
        'recall': recall_score(y_fold_val, y_pred, zero_division=0),
        'f1_score': f1_score(y_fold_val, y_pred, zero_division=0),
        'auc_roc': roc_auc_score(y_fold_val, y_proba)
    }
    
    return metrics

lakukan analisis dari 10 hasil validasi model SVM untuk satu kombinasi parameter yang sedang dilakukan eksperimen (rataan hasil dan juga standar deviasi dari hasil 10-fold cross validation). 

In [ ]:
# Kombinasi 1: kernel = linear, C = 0.1
kernel = 'linear'
C = 0.1
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")

sempat keluar hasil, tapi ketika di run hasilnya hilang, berikut output yang sempat di capture
![hasil svm](svm0_1.jpg)

In [ ]:
# Kombinasi 2: kernel = linear, C = 1
kernel = 'linear'
C = 1
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")

ini juga sempat keluar hasil tapi ketika di run ulang outputnya hilang 
![hasil svm](svm1.jpg)

In [ ]:
# Kombinasi 3: kernel = linear, C = 10
kernel = 'linear'
C = 10
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")

HASIL TIDAK KELUAR SETELAH 600 MENIT LEBIH! 
![hasil svm](svm10.jpg)

In [ ]:
# Kombinasi 4: kernel = linear, C = 100
kernel = 'linear'
C = 100
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")


Evaluating SVM with kernel=linear and C=10


In [ ]:
# Kombinasi 5: kernel = linear, C = 1000
kernel = 'linear'
C = 1000
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")

In [ ]:
# Kombinasi 6: kernel = rbf, C = 0.1
kernel = 'rbf'
C = 0.1
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")

In [ ]:
# Kombinasi 7: kernel = linear, C = 1
kernel = 'rbf'
C = 1
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")

In [ ]:
# Kombinasi 8: kernel = linear, C = 10
kernel = 'rbf'
C = 10
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")

In [ ]:
# Kombinasi 9: kernel = rbf, C = 100
kernel = 'rbf'
C = 100
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")

In [ ]:
# Kombinasi 10: kernel = linear, C = 1000
kernel = 'rbf'
C = 1000
print(f"\nEvaluating SVM with kernel={kernel} and C={C}")
fold_metrics = Parallel(n_jobs=-1)(
    delayed(evaluate_svm)(kernel, C, train_idx, val_idx, X_train, y_train)
    for train_idx, val_idx in cv.split(X_train, y_train)
)
fold_metrics_df = pd.DataFrame(fold_metrics)
print(f"Mean metrics:\n{fold_metrics_df.mean()}")
print(f"Std metrics:\n{fold_metrics_df.std()}")

tentukan kombinasi parameter yang dianggap merupakan model terbaik. Kemudian dengan menggunakan nilai kombinasi parameter terebut, jalankan model SVM pada keseluruhan data pelatihan (fit the model) dan hitung kinerja (akurasi, presisi, recall, f1-score, dan AUC ROC) dari model

In [ ]:
#Cari model terbaik berdasarkan mean f1-score
results_df = pd.DataFrame(results)
best_model_info = results_df.sort_values(by='mean_f1_score', ascending=False).iloc[0]
print("\nModel terbaik berdasarkan hasil cross-validation:")
print(best_model_info)

In [ ]:
#Fit model terbaik ke seluruh data train
best_kernel = best_model_info['kernel']
best_C = best_model_info['C']

final_svm_model = SVC(kernel=best_kernel, C=best_C, probability=True, random_state=42)
final_svm_model.fit(X_train, y_train)

In [ ]:
#Evaluasi di data train
y_train_pred = final_svm_model.predict(X_train)
y_train_proba = final_svm_model.predict_proba(X_train)[:,1]

print("\nEvaluasi model terbaik pada seluruh data train:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred, zero_division=0)}")
print(f"Recall: {recall_score(y_train, y_train_pred, zero_division=0)}")
print(f"F1 Score: {f1_score(y_train, y_train_pred, zero_division=0)}")
print(f"AUC ROC: {roc_auc_score(y_train, y_train_proba)}")

gunakan model SVM terbaik untuk memprediksi semua record dari data tes (predict the model). Hitung akurasi, presisi, recall, f1-score, dan AUC ROC dari model terbaik pada data tes (data yang sama sekali tidak digunakan dalam membangun model terbaik). 

In [ ]:
#Prediksi pada data test
y_test_pred = final_svm_model.predict(X_test)
y_test_proba = final_svm_model.predict_proba(X_test)[:,1]

print("\nEvaluasi model terbaik pada data test:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(f"Precision: {precision_score(y_test, y_test_pred, zero_division=0)}")
print(f"Recall: {recall_score(y_test, y_test_pred, zero_division=0)}")
print(f"F1 Score: {f1_score(y_test, y_test_pred, zero_division=0)}")
print(f"AUC ROC: {roc_auc_score(y_test, y_test_proba)}")